# Numerical Methods for Manu Body Physics, Assignment #3

Yoav Zack, ID 211677398

In [ ]:
using LinearAlgebra
using ITensors
using Statistics
using Plots
using LsqFit
using Printf
theme(:dracula)

ITensors.enable_debug_checks()

## Question 1: $J_1 - J_2$ Spin-1/2 Chain

We will use the `ITensors.jl` library to generate a Heisenberg Hamiltonian with general finite $J_1,J_2$ terms:

In [ ]:
function heisenbergH(sites, g::Real)
    N = length(sites)
    
    os = OpSum()
    for j in 1:N-2
        os += 1/2,"S+",j,"S-",j+1
        os += 1/2,"S-",j,"S+",j+1
        os += "Sz",j,"Sz",j+1
        
        os += 1/2*g,"S+",j,"S-",j+2
        os += 1/2*g,"S-",j,"S+",j+2
        os += g,"Sz",j,"Sz",j+2
    end
    os += 1/2,"S+",N-1,"S-",N
    os += 1/2,"S-",N-1,"S+",N
    os += "Sz",N-1,"Sz",N
    
    H = MPO(os, sites)
    return H
end

We will use the DMRG method on this Hamiltonian to get the ground state and the 1st excited state. We will loop over system sizes, for each defining 
`sitetinds` object and the Hamiltonian, and for each finding the energy of the ground state with $S_z=0$ and 1st excited state with $S_z=1$:

In [ ]:
gc = 0.241
glist = [0.1, 0.2, 0.45, 0.49]
Nlist = 2 .^ (3:6)

nsweeps = 5
cutoff = [1E-10]
maxM = maximum(Nlist)

E0arr = zeros(length(glist), length(Nlist))
E1arr = zeros(length(glist), length(Nlist))

psi0list = Array{Any}(undef, length(glist), length(Nlist))
psi1list = Array{Any}(undef, length(glist), length(Nlist))

Threads.@threads for gind in 1:length(glist)
    g = glist[gind]
    for (Nind, N) in enumerate(Nlist)
        @show g, N
        
        sites = siteinds("S=1/2", N; conserve_qns=true)
        H = heisenbergH(sites, g);
    
        state = [isodd(n) ? "Up" : "Dn" for n=1:N]
        psi0 = MPS(sites,state);
        state[2] = "Up"
        psi1 = MPS(sites,state);
    
        E0arr[gind, Nind], psi0list[gind, Nind] = dmrg(H,psi0; nsweeps=nsweeps, cutoff=cutoff, maxdim=maxM, outputlevel=0);
        E1arr[gind, Nind], psi1list[gind, Nind] = dmrg(H,psi1; nsweeps=nsweeps, cutoff=cutoff, maxdim=maxM, outputlevel=0);
    end
end

To verify the expected behaviour, we will fit a parabolic relation to the gap:

In [ ]:
@. parabola_model(x, a) = a[1] * x^2 + a[2]
@. linear_model(x, a) = a[1] * x + a[2]
x = 1 ./ Nlist
xh = LinRange(0, 1.1/minimum(Nlist), 100)

intercept = []

plt = plot()
for (gind, g) in enumerate(glist)
    y = E1arr[gind,:] .- E0arr[gind,:]
    model = (g>gc) ? parabola_model : linear_model
    fitobj = curve_fit(model, x, y, [1.0, 0.0])
    @show coef(fitobj)
    scatter!(x, y, label=@sprintf("g = %1.3f", g), color=palette(:default)[gind])
    plot!(xh, model(xh, coef(fitobj)), label=nothing, color=palette(:default)[gind])
    push!(intercept, coef(fitobj)[1])
end
plot!(xlims=(0, Inf), ylims=(0, Inf))
xlabel!("1/N")
ylabel!("ΔE")

And we can clearly see that for $g<g_c$ the system's gap goes to 0 at $N\to \infty$, but for $g>g_c$ the system is gapped even in $N\to\infty$. Now we will calculate the correlations for different values of $g$ in the longest chain we checked before:

In [ ]:
ps = []
corr = []
for (gind, g) in enumerate(glist)
    xmax = Int(maximum(Nlist)/2)
    x = 1:xmax
    x = (g<gc) ? log.(x) : x
    y = log.(abs.(correlation_matrix(psi0list[gind, length(Nlist)],"Sz","Sz")[1:xmax,1]))
    push!(corr, y)
    p = plot(x, corr[gind], label=@sprintf("g=%.2f",g), ylabel="log[corr]", xlabel=(g<gc) ? "Log[Distance]" : "Distance")
    push!(ps, p)
end
plot(ps..., plot_title="Sz-Sz Correlations")

And as expected the $g<g_c$ cases are linear in a log-log plot (i.e. they are power laws) while the cases for $g>g_c$ are linear in a semilog-y plot (i.e. they decay exponentially). We will estimate the power of the power laws and the correlation length of the exponential laws using linear fits:

In [ ]:
for (gind, g) in enumerate(glist)
    if ps[gind].n > 1
        continue # already plotted fit
    end
    y = corr[gind]
    x = 1:length(y)
    x = (g<gc) ? log.(x) : x # fix x axis for power laws or exponential decay
    fitobj = curve_fit(linear_model, x, y, [1.0, 0.0])
    fit_label = (g<gc) ? @sprintf("α=%.2f", coef(fitobj)[1]) : @sprintf("ζ=%.2f", 1.0/coef(fitobj)[1])
    p = plot(ps[gind], x, linear_model(x, coef(fitobj)), label=fit_label, ylabel="log[corr]", xlabel=(g<gc) ? "Log[Distance]" : "Distance")
    ps[gind] = p
end
plot(ps..., plot_title="Linear Fit to Correlation Decay")

So the value for $\alpha$ is always about $\alpha \approx 1$, while the value for $\zeta$ changes based on $g$ but is of the order of unity.